# Ground state structure optimization

Molecular structure optimizations always start from an initial guess of the molecular geometry (constructed using chemistry software, e.g. Avogadro, or downloaded from a database). In the following we will perform a ground state structure optimization of cafestol and kahweol. Cafestol is a molecule contained by Robusta coffee beans, while kahweol is a very similar molecule contained by Arabica coffee beans. Kahweol has a distinctive Raman peak which allows to identify Arabica coffee beans by Raman spectroscopy. Here, we determine the relaxed geometries of these two molecules at different levels of theory, to be used in [Raman spectroscopy calculations](sec:raman-tutorial).

In [2]:
import veloxchem as vlx
import gator
import py3Dmol as p3d
import sys

In [3]:
cafestol_xyz = """51
cafestol initial structure
O  -4.5215   -1.3455    0.4926
O  -5.2905    0.0812   -1.7033
O   5.1630    0.4345   -0.1101
C  -1.2807   -0.7511   -0.0465
C  -0.5244    0.5810   -0.4102
C   0.9909    0.6610    0.0407
C  -2.9440    0.2916    1.2866
C  -1.8143   -0.7311    1.3945
C  -2.6188   -0.8159   -0.8642
C  -3.7116   -0.2387    0.0606
C  -0.4332   -2.0024   -0.3283
C  -1.3742    1.8303   -0.0654
C   1.6844   -0.6873   -0.4019
C  -2.3206    1.6833    1.1309
C   0.9997   -1.9155    0.1913
C   1.7285    1.8344   -0.7277
C   1.1437    0.9261    1.5588
C   3.1644   -0.6155   -0.2085
C  -4.6490    0.7442   -0.6282
C   3.2525    1.9378   -0.4590
C   3.8298    0.5879   -0.2856
C   4.1654   -1.5888    0.0140
C   5.3604   -0.9015    0.0689
H  -0.4604    0.5817   -1.5108
H  -3.5646    0.2626    2.1897
H  -1.0861   -0.4449    2.1498
H  -2.1902   -1.7145    1.7003
H  -2.5286   -0.2495   -1.7981
H  -2.8534   -1.8505   -1.1460
H  -0.3943   -2.1694   -1.4136
H  -0.9135   -2.8938    0.0946
H  -1.9810    2.0650   -0.9487
H  -0.7587    2.7222    0.0818
H   1.5605   -0.7646   -1.4951
H  -1.7642    1.9154    2.0477
H  -3.1082    2.4431    1.0635
H   1.0176   -1.9046    1.2859
H   1.5276   -2.8282   -0.1118
H   1.2808    2.8061   -0.4944
H   1.5851    1.6842   -1.8068
H   0.5898    1.8173    1.8670
H   0.8147    0.0892    2.1727
H   2.1832    1.0992    1.8532
H  -4.1337    1.6158   -1.0351
H  -5.4265    1.0934    0.0595
H   3.4273    2.5264    0.4487
H   3.7385    2.4646   -1.2871
H  -3.9519   -2.0832    0.7635
H   4.0442   -2.6573    0.1180
H  -5.7603   -0.6874   -1.3370
H   6.3869   -1.2031    0.2186
"""
cafestol = vlx.Molecule.from_xyz_string(cafestol_xyz)
basis_set_label = "sto-3g" # still too big to run analytical gradient with pyscf derivatives
cafestol_basis = vlx.MolecularBasis.read(cafestol, basis_set_label)

In [4]:
kahweol_xyz = """49
kahweol initial structure
O  -4.4710   -1.5361    0.0905
O  -5.6188    0.9753   -0.0811
O   5.1625    0.5695   -0.0997
C  -1.2209   -0.8317   -0.2017
C  -0.4821    0.5357   -0.4703
C  -3.0038    0.1167    1.0462
C  -1.8514   -0.8737    1.1997
C  -2.4967   -0.9058   -1.1119
C   0.9884    0.6333    0.0944
C  -3.6676   -0.3924   -0.2474
C  -0.3220   -2.0551   -0.4569
C  -1.3919    1.7479   -0.1363
C  -2.4167    1.5304    0.9816
C   1.7435   -0.6786   -0.3542
C   1.0741   -1.9483    0.1552
C   1.0367    0.8330    1.6293
C  -4.5845    0.5870   -0.9677
C   1.7610    1.8511   -0.4907
C   3.2146   -0.5444   -0.1101
C   3.1013    1.8722   -0.6493
C   3.8167    0.6946   -0.3059
C   4.2309   -1.4546    0.2464
C   5.3980   -0.7259    0.2399
H  -0.3415    0.5842   -1.5631
H  -3.6779    0.0397    1.9072
H  -1.1861   -0.6021    2.0159
H  -2.2356   -1.8757    1.4322
H  -2.3648   -0.3090   -2.0216
H  -2.6844   -1.9384   -1.4328
H  -0.2098   -2.1924   -1.5413
H  -0.8043   -2.9694   -0.0890
H  -1.9406    2.0033   -1.0514
H  -0.8134    2.6482    0.0921
H  -1.9341    1.7452    1.9434
H  -3.2193    2.2709    0.8849
H   1.6699   -0.7251   -1.4553
H   1.0259   -1.9792    1.2481
H   1.6508   -2.8307   -0.1490
H   2.0514    1.0518    1.9842
H   0.4166    1.6762    1.9486
H   0.7233   -0.0489    2.1864
H  -5.0524    0.1195   -1.8415
H  -4.0679    1.4861   -1.3088
H   1.2088    2.7568   -0.7218
H  -5.0925   -1.2746    0.7905
H   3.6240    2.7536   -0.9997
H   4.1386   -2.5085    0.4650
H  -6.2132    0.2149    0.0339
H   6.4308   -0.9731    0.4374
"""
kahweol = vlx.Molecule.from_xyz_string(kahweol_xyz)
kahweol_basis = vlx.MolecularBasis.read(kahweol, basis_set_label)

The difference between cafestol and kahweol is very subtle, essentially only one double bond and two H atoms.

In [10]:
print('Cafestol, starting geometry:')
vc = p3d.view(500,200)
vc.addModel(cafestol_xyz, 'xyz')
vc.setStyle({'stick': {}})
vc.zoomTo()
vc.show()
print('Kahweol, starting geometry:')
vk = p3d.view(500,200)
vk.addModel(kahweol_xyz, 'xyz')
vk.setStyle({'stick': {}})
vk.zoomTo()
vk.show()

Cafestol, starting geometry:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Kahweol, starting geometry:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## XTB geometry optimization
```python
# Set up the xtb driver
method_settings = {'xtb':'gfn2'}
ostream = vlx.OutputStream(sys.stdout)
xtbdrv = vlx.XTBDriver()
xtbdrv.set_method(method_settings['xtb'].lower())
xtbdrv.compute(cafestol, ostream)
```

                                                                                                                              
                                                            XTB Driver                                                        
                                                           ============                                                       
                                                                                                                              
    * Info *   Energy   : -69.3615122026 a.u.                                                                                 
    * Info *   Gradient : 8.325170e-03 a.u. (RMS)                                                                             
    * Info *              1.928275e-02 a.u. (Max)                                                                             
    * Info *   Time     : 0.12 sec                                                                                            
                                                                                                                              
    * Info * Reference: C. Bannwarth, E. Caldeweyher, S. Ehlert,                                                              
    * Info * A. Hansen, P. Pracht, J. Seibert, S. Spicher, S. Grimme,                                                         
    * Info * WIREs Comput. Mol. Sci., 2020, 11, e01493
    ...

### Set up the xtb gradient and optimization drivers and run the geometry optimization

```python
xtb_grad_drv = vlx.XTBGradientDriver(xtbdrv)
xtb_opt_drv = vlx.OptimizationDriver(xtb_grad_drv, flag='xtb')
xtb_opt_cafestol = xtb_opt_drv.compute(cafestol, cafestol_basis)

ostream.print_block(xtb_opt_cafestol.get_string())
ostream.flush()
```

                                                                                                                              
                                                    Optimization Driver Setup                                                 
                                                   ===========================                                                
                                                                                                                              
                                         Coordinate System       :    TRIC                                                    
                                         Constraints             :    No                                                      
                                         Max. Number of Steps    :    300 
                                                                                       
                                                                                                                              
                                         ...
                                         ...
                                         ...
 

    Step   19 : Displace = [92m3.403e-04[0m/[92m8.142e-04[0m (rms/max) Trust = 3.000e-01 (=) Grad = [92m8.272e-06[0m/[92m2.364e-05[0m (rms/max) E (change) = -69.3730867918 ([92m-5.629e-08[0m) Quality = [0m1.322[0m
    Hessian Eigenvalues: 1.03149e-03 3.17540e-03 5.00378e-03 ... 5.32393e-01 5.90119e-01 6.49493e-01
    Converged! =D
    
        #==========================================================================#
        #| If this code has benefited your research, please support us by citing: |#
        #|                                                                        |#
        #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
        #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
        #| http://dx.doi.org/10.1063/1.4952956                                    |#
        #==========================================================================#
        Time elapsed since start of run_optimizer: 10.922 seconds


    * Info * Geometry optimization completed.

...
                                              
                                                 Summary of Geometry Optimization                                             
                                                ==================================                                            
                                                                                                                              
                      Opt.Step       Energy (a.u.)       Energy Change (a.u.)       Displacement (RMS, Max)                   
                      -------------------------------------------------------------------------------------                   
                          0         -69.361512203479        0.000000000000         0.000e+00      0.000e+00                   
                          1         -69.371005882560       -0.009493679081         9.371e-02      1.764e-01                   
                          2         -69.372246054312       -0.001240171752         3.111e-02      6.262e-02                   
                          3         -69.372642766329       -0.000396712016         2.276e-02      5.492e-02                   
                          4         -69.372862573937       -0.000219807609         1.641e-02      3.634e-02                   
                          5         -69.372957708359       -0.000095134422         1.237e-02      2.656e-02                   
                          6         -69.373032070669       -0.000074362310         1.313e-02      3.164e-02                   
                          7         -69.373067959045       -0.000035888375         1.260e-02      3.281e-02                   
                          8         -69.373075372303       -0.000007413258         3.832e-03      1.119e-02                   
                          9         -69.373080412780       -0.000005040477         2.595e-03      6.016e-03                   
                         10         -69.373083029538       -0.000002616758         3.013e-03      7.562e-03                   
                         11         -69.373083940650       -0.000000911113         1.280e-03      2.691e-03                   
                         12         -69.373084792743       -0.000000852092         1.673e-03      4.493e-03                   
                         13         -69.373085263501       -0.000000470758         9.108e-04      3.119e-03                   
                         14         -69.373085745359       -0.000000481858         1.157e-03      3.994e-03                   
                         15         -69.373086127854       -0.000000382495         1.124e-03      3.963e-03                   
                         16         -69.373086401837       -0.000000273982         9.914e-04      3.328e-03                   
                         17         -69.373086606283       -0.000000204446         8.354e-04      2.813e-03                   
                         18         -69.373086735508       -0.000000129225         6.379e-04      1.836e-03                   
                         19         -69.373086791798       -0.000000056290         3.403e-04      8.142e-04                   
                                                                                                                              
                                                                                                                              
                                                  Statistical Deviation between                                               
                                             Optimized Geometry and Initial Geometry                                          
                                            =========================================                                         
                                                                                                                              
                                   Internal Coord.        RMS deviation         Max. deviation                                
                                   -----------------------------------------------------------                                
                                      Bonds               0.011 Angstrom        0.036 Angstrom                                
                                      Angles              1.350 degree          3.923 degree                                  
                                      Dihedrals           3.196 degree         13.587 degree                                  
                                                                                                                              
                                         *** Time spent in Optimization Driver: 11.34 sec               

### Run the XTB structure optimization for kahweol:

```python
xtbdrv = vlx.XTBDriver()
xtbdrv.set_method(method_settings['xtb'].lower())
xtbdrv.compute(kahweol, ostream)
xtb_grad_drv = vlx.XTBGradientDriver(xtbdrv)
xtb_opt_drv = vlx.OptimizationDriver(xtb_grad_drv, flag='xtb')
xtb_opt_kahweol = xtb_opt_drv.compute(kahweol, kahweol_basis)
ostream.print_block(xtb_opt_kahweol.get_string())
ostream.flush()
```

                                                                                                                              
                                                            XTB Driver                                                        
                                                           ============                                                       
                                                                                                                              
    * Info *   Energy   : -68.3102341570 a.u.                                                                                 
    * Info *   Gradient : 8.430853e-03 a.u. (RMS)                                                                             
    * Info *              3.152908e-02 a.u. (Max)                                                                             
    * Info *   Time     : 0.09 sec                                                                                            
                                                                                                                              
    * Info * Reference: C. Bannwarth, E. Caldeweyher, S. Ehlert,                                                              
    * Info * A. Hansen, P. Pracht, J. Seibert, S. Spicher, S. Grimme,                                                         
    * Info * WIREs Comput. Mol. Sci., 2020, 11, e01493                                                                        
                                                                                                                              
                                                                                                                              
                                                    Optimization Driver Setup                                                 
                                                   ===========================                                                
                                                                                                                              
                                         Coordinate System       :    TRIC                                                    
                                         Constraints             :    No                                                      
                                         Max. Number of Steps    :    300 
                                         ...
                                         ...
                                         ...
                                                 Summary of Geometry Optimization                                             
                                                ==================================                                            
                                                                                                                              
                      Opt.Step       Energy (a.u.)       Energy Change (a.u.)       Displacement (RMS, Max)                   
                      -------------------------------------------------------------------------------------                   
                          0         -68.310234157198        0.000000000000         0.000e+00      0.000e+00                   
                          1         -68.319437730841       -0.009203573643         9.448e-02      2.163e-01                   
                          2         -68.320707883896       -0.001270153055         3.422e-02      7.343e-02                   
                          3         -68.321101833645       -0.000393949749         3.393e-02      7.122e-02                   
                          4         -68.321262321180       -0.000160487535         1.908e-02      4.656e-02                   
                          5         -68.321347171693       -0.000084850512         2.113e-02      5.546e-02                   
                          6         -68.321403406885       -0.000056235192         6.867e-03      1.655e-02                   
                          7         -68.321428918243       -0.000025511358         7.166e-03      2.091e-02                   
                          8         -68.321447664121       -0.000018745878         7.271e-03      2.488e-02                   
                          9         -68.321467602202       -0.000019938081         7.604e-03      2.985e-02                   
                         10         -68.321489028322       -0.000021426120         1.267e-02      5.192e-02                   
                         11         -68.321495961945       -0.000006933624         5.653e-03      2.341e-02                   
                         12         -68.321500842839       -0.000004880894         5.779e-03      2.311e-02                   
                         13         -68.321503105894       -0.000002263054         2.534e-03      9.698e-03                   
                         14         -68.321504562101       -0.000001456207         1.622e-03      5.554e-03                   
                         15         -68.321505303040       -0.000000740939         1.216e-03      3.786e-03                   
                         16         -68.321505603135       -0.000000300096         8.057e-04      2.964e-03                   
                         17         -68.321505896886       -0.000000293750         1.067e-03      4.209e-03                   
                         18         -68.321506132000       -0.000000235114         9.041e-04      3.413e-03                   
                         19         -68.321506307525       -0.000000175525         7.735e-04      2.782e-03                   
                         20         -68.321506377051       -0.000000069525         3.570e-04      1.372e-03                   
                                                                                                                              
                                                                                                                              
                                                  Statistical Deviation between                                               
                                             Optimized Geometry and Initial Geometry                                          
                                            =========================================                                         
                                                                                                                              
                                   Internal Coord.        RMS deviation         Max. deviation                                
                                   -----------------------------------------------------------                                
                                      Bonds               0.012 Angstrom        0.033 Angstrom                                
                                      Angles              1.359 degree          3.480 degree                                  
                                      Dihedrals           3.908 degree         13.103 degree                                  
                                                                                                                              
                                         *** Time spent in Optimization Driver: 11.60 sec                                  

### Compare initial and final geometries

In [11]:
def xyz(molecule):
    """
    Returns the molecular geometry in xyz format.
    """
    from veloxchem.veloxchemlib import ChemicalElement
    from veloxchem.veloxchemlib import bohr_in_angstroms

    elem_ids = molecule.elem_ids_to_numpy()

    xs = molecule.x_to_numpy() * bohr_in_angstroms()
    ys = molecule.y_to_numpy() * bohr_in_angstroms()
    zs = molecule.z_to_numpy() * bohr_in_angstroms()

    txt = "" 
    txt +="%d\n\n" % molecule.number_of_atoms()
    for elem_id, x, y, z in zip(elem_ids, xs, ys, zs):
        elem = ChemicalElement()
        elem.set_atom_type(elem_id)
        txt += "%6s %22.12f %22.12f %22.12f\n" % (elem.get_name(),x,y,z)
        
    return txt

In [12]:
# xtb_opt_cafestol.write_xyz("xtb_opt_cafestol.xyz")
# xtb_opt_kahweol.write_xyz("xtb_opt_kahweol.xyz")
# 
xyz_xtb_opt_cafestol = open("xtb_opt_cafestol.xyz", "r").read() # read optimized geometry from file
xyz_xtb_opt_kahweol = open("xtb_opt_cafestol.xyz", "r").read()

print('Cafestol, starting geometry vs. XTB optimized geometry:')
vc = p3d.view(width=500,height=200)
vc.addModel(cafestol_xyz, 'xyz')
vc.addModel(xyz_xtb_opt_cafestol, 'xyz')
vc.setStyle({'stick': {}})
vc.zoomTo()
vc.show()

print('Kahweol, starting geometry vs. XTB optimized geometry:')
vk = p3d.view(width=500,height=200)
vk.addModel(kahweol_xyz, 'xyz')
vk.addModel(xyz_xtb_opt_kahweol, 'xyz')
vk.setStyle({'stick': {}})
vk.zoomTo()
vk.show()

Cafestol, starting geometry vs. XTB optimized geometry:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

Kahweol, starting geometry vs. XTB optimized geometry:


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

## SCF geometry optimization
```python
# Set up the scf driver
method_settings = {} # HF or DFT (add, e.g. 'xcfun':'b3lyp' to run a B3LYP calculation)
scf_settings = {'conv_thresh': 1.0e-5}
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(cafestol, cafestol_basis) # ! using sto-3g here...
```

                                                                                                                              
                                                Self Consistent Field Driver Setup                                            
                                               ====================================                                           
                                                                                                                              
                       Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                       Initial Guess Model             : Superposition of Atomic Densities                                    
                       Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                       Max. Number of Iterations       : 50                                                                   
                       Max. Number of Error Vectors    : 10                                                                   
                       Convergence Threshold           : 1.0e-05                                                              
                       ERI Screening Scheme            : Cauchy Schwarz + Density                                             
                       ERI Screening Mode              : Dynamic                                                              
                       ERI Screening Threshold         : 1.0e-12                                                              
                       Linear Dependence Threshold     : 1.0e-06  
                       ...
                       ...
                       ...

### Run SCF geometry optimization for cafestol
```python
gradient_settings = {'numerical':'no'} # use analytical SCF gradient
scf_grad_drv = vlx.ScfGradientDriver(scfdrv)
scf_grad_drv.update_settings(gradient_settings, method_settings)
scf_opt_drv = vlx.OptimizationDriver(scf_grad_drv, flag='scf')
scf_opt_cafestol = scf_opt_drv.compute(cafestol, cafestol_basis)
```

                                                                                                                              
                                                    Optimization Driver Setup                                                 
                                                   ===========================                                                
                                                                                                                              
                                         Coordinate System       :    TRIC                                                    
                                         Constraints             :    No                                                      
                                         Max. Number of Steps    :    300                                                     
                                          
                                          ...
                                          ...
                                          ...

### Run SCF geometry optimization for kahweol
```python
scfdrv = vlx.ScfRestrictedDriver()
scfdrv.update_settings(scf_settings, method_settings)
scfdrv.compute(kahweol, kahweol_basis) # ! using sto-3g here...
scf_grad_drv = vlx.ScfGradientDriver(scfdrv)
scf_grad_drv.update_settings(gradient_settings, method_settings)
scf_opt_drv = vlx.OptimizationDriver(scf_grad_drv, flag='scf')
scf_opt_kahweol = scf_opt_drv.compute(kahweol, kahweol_basis)
```

                                                                                                                              
                                                Self Consistent Field Driver Setup                                            
                                               ====================================                                           
                                                                                                                              
                       Wave Function Model             : Spin-Restricted Hartree-Fock                                         
                       Initial Guess Model             : Superposition of Atomic Densities                                    
                       Convergence Accelerator         : Two Level Direct Inversion of Iterative Subspace                     
                       Max. Number of Iterations       : 50                                                                   
                       Max. Number of Error Vectors    : 10                                                                   
                       Convergence Threshold           : 1.0e-05                                                              
                       ERI Screening Scheme            : Cauchy Schwarz + Density                                             
                       ERI Screening Mode              : Dynamic                                                              
                       ERI Screening Threshold         : 1.0e-12                                                              
                       Linear Dependence Threshold     : 1.0e-06
                       ...
                       ...
                       ...
                                                                     
                                                    Optimization Driver Setup                                                 
                                                   ===========================                                                
                                                                                                                              
                                         Coordinate System       :    TRIC                                                    
                                         Constraints             :    No                                                      
                                         Max. Number of Steps    :    300                                                     
                                                                           
                       ...
                       ...
                       ...
                       

## MP2 geometry optimization (Gator)


In [ ]:
# TODO: add